In [1]:
import numpy as np
import random
np.random.seed(87)
random.seed(87)

In [2]:
test = np.load('test.npz')
train = np.load('train.npz')

In [3]:
list(test.keys())

['image', 'label']

In [4]:
test['label']
np.unique(test['label'])

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [5]:
y = test['label']

np.zeros((y.size, 10)).shape
np.arange(y.size)

new_y = np.eye(10)[[int(y_) for y_ in y.tolist()]]
new_y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [6]:
y_test = test['label']
X_test = test['image']
y_train = train['label']
X_train = train['image']

# transfer to one-hot vector
#y_onehot_train = np.zeros((y_train.size, 10))
#y_onehot_train[np.arange(y_train.size), y_train] = 1
#y_onehot_test = np.zeros((y_test.size, 10))
#y_onehot_test[np.arange(y_test.size), y_train] = 1
y_onehot_train = np.eye(10)[[int(y_) for y_ in y_train.tolist()]]
y_onehot_test = np.eye(10)[[int(y_) for y_ in y_test.tolist()]]


y_train = np.moveaxis(y_onehot_train, 1, 0)
print(X_train.shape)
X_train = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
print(X_train.shape)
X_train = np.moveaxis(X_train, 1, 0)
print(X_train.shape)

y_test = np.moveaxis(y_onehot_test, 1, 0)
X_test = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])
X_test = np.moveaxis(X_test, 1, 0)

(12000, 28, 28)
(12000, 784)
(784, 12000)


In [7]:
# X_train[X_train < 133] = -1
# # X_train[X_train < 133] = 0

# X_train[X_train >= 133] = 1

X_train /= 225
X_test /= 225

In [8]:
y_test[:, 0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [9]:
np.unique(test['image'])

array([  0.,  11.,  16.,  17.,  46.,  47.,  52.,  53.,  54.,  59.,  60.,
        89.,  90.,  95.,  96.,  97., 102., 103., 132., 133., 138., 139.,
       140., 145., 146., 152., 175., 176., 181., 182., 183., 188., 189.,
       218., 219., 224., 225.])

In [10]:
np.unique(train['image'])

array([  0.,  11.,  16.,  17.,  46.,  47.,  52.,  53.,  54.,  59.,  60.,
        89.,  90.,  95.,  96.,  97., 102., 103., 132., 133., 138., 139.,
       140., 145., 146., 175., 176., 181., 182., 183., 188., 189., 218.,
       219., 224., 225.])

In [11]:
np.unique(train['image'])[len(np.unique(train['image']))//2]

132.0

In [12]:
class Layer:
    def __init__(self, last_layer_num, node_num, activation_function=None, custom_W=None, custom_b=None):
        self.__node_num = node_num  #output_dim
        self.__last_layer_num = last_layer_num  #input_dim

        
        self.__layer_input = None
        self.activation = activation_function
        
        if custom_W is not None:
            self.W_ = custom_W
        else:
#             limit = 1 / np.sqrt(self.__last_layer_num)
#             self.W_ = np.random.uniform(-limit, limit, (self.__last_layer_num, self.__node_num))
            self.W_ = np.zeros((self.__last_layer_num, self.__node_num))
        
        if custom_b is not None:
            self.b_ = custom_b
        else:
            self.b_ = np.zeros((self.__node_num, 1))

    def forward_propagation(self, last_layer):
        self.__layer_input = last_layer
        if self.activation:
            return self.activation(np.dot(self.W_.T, last_layer) + self.b_)
        else:
            return np.dot(self.W_.T, last_layer) + self.b_
        
    def back_propagation(self, gradient, learning_rate = 0.03, gamma = None):
        """ If gamma is not None, use RMSprop"""
        if self.activation:
            gradient = self.activation.gradient(gradient)
        
        W_temp = self.W_
        
        if gamma:
            sqr = np.zeros_like(W_temp)
            g = W_temp / gradient.shape[1]
            sqr[:] = gamma * sqr + (1. - gamma) * np.square(g)
            gradient_W = learning_rate * g / np.sqrt(sqr + 1e-8)
            
            gradient_b = np.sum(gradient, axis=1, keepdims=True)
            self.W_ = self.W_ - gradient_W
            assert self.W_.shape == gradient_W.shape
            self.b_ = self.b_ - learning_rate * gradient_b
            assert self.b_.shape == gradient_b.shape
        else:
            gradient_W = self.__layer_input.dot(gradient.T)
            gradient_b = np.sum(gradient, axis=1, keepdims=True)

            #print("gradient : ",gradient)
            #print("gradient_W : ",gradient_W)
            self.W_ = self.W_ - learning_rate * gradient_W
            assert self.W_.shape == gradient_W.shape
            self.b_ = self.b_ - learning_rate * gradient_b
            assert self.b_.shape == gradient_b.shape
           # print("self.W_:",self.W_[1,1])
        
        accumulated_gradient = W_temp.dot(gradient)
        return accumulated_gradient

In [13]:
# test forward propagation

W = np.array([[1, 2, 3], [1, 2, 3]]).T
b = np.array([0, 1])
data = np.array([[3, 4, 5],[3, 4, 5]]).T

layer = Layer(3, 2, None, W, b)
layer1_output = layer.forward_propagation(data)

print(layer1_output)

layer2 = Layer(2, 3)
print(layer2.W_)
layer2.forward_propagation(layer1_output)

[[26 27]
 [26 27]]
[[0. 0. 0.]
 [0. 0. 0.]]


array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [14]:
def Softmax(x):
    e_x = np.exp(x - np.max(x))  # 減max會比較穩定,避免overflow
    return e_x / e_x.sum(axis=0)  # -xmax會消掉

In [15]:
class Sigmoid():
    def __call__(self,x):
        # To prevent from overflow
        x = np.clip(x, 1e-15, 1 - 1e-15)
        return 1.0 / (1.0 + np.exp(-x))
    def gradient(self, x):
        return self(x) * (1.0 - self(x))

In [16]:
class ReLU():
    def __call__(self, x):
        return np.maximum(x, 0.0)
    
    def gradient(self, x):
        x[x <= 0] = 0.0
        x[x > 0] = 1.0
        return x


In [17]:
class CrossEntropy():
    def __call__(self, y_hat, y):
        # Avoid division by zero
        y_hat = np.clip(y_hat, 1e-15, 1 - 1e-15)
        # return
        loss_sum = 0
        for n in range(y.shape[0]):
            for k in range(y.shape[1]):
                if y[n, k] == 1:
                    loss_sum += -np.log(y_hat[n, k])
        loss = np.sum(loss_sum) / y.shape[0]
        return loss

    def gradient(self, y_hat, y):
        return y_hat - y

In [18]:
class CrossEntropy2():
    def __call__(self, y_hat, y):
        # Avoid division by zero
        y_hat = np.clip(y_hat, 1e-15, 1 - 1e-15)
        # softmax
        m = y.shape[0]
        p = Softmax(y_hat)
        y = np.argmax(y, axis=1)
        
        log_likelihood = -np.log(p[range(m), y])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def gradient(self, y_hat, y):
        m = y.shape[0]
#         grad = Softmax(y_hat)
        grad = y_hat
        y = np.argmax(y, axis=1)
        grad[range(m), y] -= 1
        grad = grad / m
        return grad
#         return y_hat - y

In [19]:
 # test 
y = np.array([[1, 0 , 0], [1, 0, 0], [0 , 1, 0]])
y_hat = np.array([[3, 2, 2], [2, 5 ,6], [4 ,2 ,6]])

CrossEntropy2()(y_hat, y)


1.0986122886681098

In [20]:
np.argmax(y, axis=1)

array([0, 0, 1])

In [21]:
# HiddenLayer = Layer(last_layer_num=784, node_num=392, activation_function=Sigmoid())
# HiddenLayer2 = Layer(last_layer_num=392, node_num=198, activation_function=Sigmoid())
# HiddenLayer3 = Layer(last_layer_num=198, node_num=2, activation_function=Sigmoid())
# OutputLayer = Layer(last_layer_num=2, node_num=10)
# loss_func = CrossEntropy()
loss_func = CrossEntropy2()



HiddenLayer = Layer(last_layer_num=784, node_num=400, activation_function=ReLU())
HiddenLayer2 = Layer(last_layer_num=400, node_num=400, activation_function=ReLU())
HiddenLayer3 = Layer(last_layer_num=400, node_num=400, activation_function=ReLU())
OutputLayer = Layer(last_layer_num=400, node_num=10)

In [22]:
def train_epoch(X_train, y_train, epoch=10, learning_rate=1e-4):
    """ Gradient Descent """
    for i in range(epoch):
        output1 = HiddenLayer.forward_propagation(X_train)
        theta = OutputLayer.forward_propagation(output1)
        y_hat = Softmax(theta)

        loss = loss_func(y_hat, y_train)

        print(loss)
        #print(y_hat)
        
        gradient = loss_func.gradient(y_hat, y_train)

        gradient = OutputLayer.back_propagation(gradient, learning_rate=learning_rate)
        HiddenLayer.back_propagation(gradient, learning_rate=learning_rate)
    

In [23]:
#train_epoch(X_train, y_train, learning_rate=0.1)

## loss become nan

* [python - Deep-Learning Nan loss reasons - Stack Overflow](https://stackoverflow.com/questions/40050397/deep-learning-nan-loss-reasons)
* [训练深度学习网络时候，出现Nan是什么原因，怎么才能避免？ - 知乎](https://www.zhihu.com/question/49346370)

To check

* [X] Too high of a learning rate. You can often tell if this is the case if the loss begins to increase and then diverges to infinity. => we have set very small learning rate
* [X] I am not to familiar with the DNNClassifier but I am guessing it uses the categorical cross entropy cost function. This involves taking the log of the prediction which diverges as the prediction approaches zero. That is why people usually add a small epsilon value to the prediction to prevent this divergence. I am guessing the DNNClassifier probably does this or uses the tensorflow opp for it. Probably not the issue. => we have used np.clip
* [X] Other numerical stability issues can exist such as division by zero where adding the epsilon can help. Another less obvious one if the square root who's derivative can diverge if not properly simplified when dealing with finite precision numbers. Yet again I doubt this is the issue in the case of the DNNClassifier. => we have used np.clip
* [X] You may have an issue with the input data. Try calling assert not np.any(np.isnan(x)) on the input data to make sure you are not introducing the nan. Also make sure all of the target values are valid. Finally, make sure the data is properly normalized. You probably want to have the pixels in the range [-1, 1] and not [0, 255]. => we have normalized the data to [-1, 1] and make sure there is no nan
* [X] The labels must be in the domain of the loss function, so if using a logarithmic-based loss function all labels must be non-negative (as noted by evan pu and the comments below). => cross-entropy loss is a logarithmic-based loss


* [] initial weight ? => John use zeros
* [X] backpropaggtion return ? => why return accumulate_gradient ?
* [] gradient_W => node_num ?? output => input


In [24]:
def evaluation(predicts, golds):
    """ calculate accuracy """
    correct = 0
    total = len(predicts)
    assert len(predicts) == len(golds)
    for predict, gold in zip(predicts, golds):
        if predict == gold:
            correct += 1
    
    return correct / total


In [25]:
def SGD_train_epoch(X_train, y_train, batch_size=3, epoch=10, learning_rate=0.03, shuffle=True):
    """ Stochastic Gradient Descent """
    batch_num = 0
    all_loss = []
    for i in range(epoch):
        epoch_loss = []
        predicts = []
        golds = []
        
        if shuffle:
            idx = list(range(X_train.shape[1]))
            random.shuffle(idx)
            
        for start in range(0, X_train.shape[1], batch_size):
            end = min(X_train.shape[1], start + batch_size)
            
            if shuffle:
                X = X_train[:, idx[start:end]]
                y = y_train[:, idx[start:end]]
            else:
                X = X_train[:, start:end]
                y = y_train[:, start:end]
            
            #import ipdb; ipdb.set_trace()
            
            #print("y : ",y)
            output1 = HiddenLayer.forward_propagation(X)
            output2 = HiddenLayer2.forward_propagation(output1)
            output3 = HiddenLayer3.forward_propagation(output2)
            theta = OutputLayer.forward_propagation(output3)
            print("output1 : ",output1)
            
            #print("theta : ",theta)
            y_hat = Softmax(theta)
            #print("y_hat : ",y_hat)
            #print('sum:', np.sum(y_hat, axis=0))
            
            predicts += np.argmax(y_hat, axis=1).tolist()
            golds += np.argmax(y, axis=1).tolist()

            loss = loss_func(y_hat, y) / (end - start)
            #loss = loss_func(theta, y) / (end - start)
            epoch_loss.append(loss)
            
            
            
            
#             gradient = loss_func.gradient(y_hat, y) / (end - start)
            gradient = loss_func.gradient(y_hat, y)
            #print("gradient : ",gradient)
            gradient = OutputLayer.back_propagation(gradient, learning_rate=learning_rate)
            gradient = HiddenLayer3.back_propagation(gradient, learning_rate=learning_rate)
            gradient = HiddenLayer2.back_propagation(gradient, learning_rate=learning_rate)
            HiddenLayer.back_propagation(gradient, learning_rate=learning_rate)

        
       # print('Epoch:', i+1, 'Average loss:', sum(epoch_loss)/len(epoch_loss),
        #      'Accuracy:', evaluation(predicts, golds))
            
        all_loss.extend(epoch_loss)

In [ ]:
SGD_train_epoch(X_train, y_train, epoch=500)

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

...

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
output1 :  [[0. 0. 0.

In [ ]:
X_train.shape

In [ ]:
X_train[:, -4:].shape

In [ ]:
HiddenLayer.W_.shape

In [ ]:
HiddenLayer.b_.shape

In [ ]:
HiddenLayer.forward_propagation(X_train[:, -4:])

In [ ]:
y = np.array([[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [ ]:
y.shape[0]
y.size
len(y)

In [ ]:
y.shape

In [ ]:
sigmoid(3)